In [1]:
!pip install vaderSentiment
!pip install afinn
#!pip install tabula-py
#!pip install -q tabula-py
!pip install PyPDF2
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53448 sha256=bd8f4841bcbb586453a0c82423a2f2c5c4853511393fd64dffdeb27993985f13
  Stored in directory: /root/.cache/pip/wheels/79/91/ee/8374d9bc8c6c0896a2db75afdfd63d43653902407a0e76cd94
Successfully built afinn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string
#import tabula
#from tabula import read_pdf
from google.colab import drive
import PyPDF2
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
from google.colab import files
uploaded = files.upload()

Saving NEULAND.xlsx to NEULAND.xlsx


In [4]:
import io
concall = pd.read_excel(io.BytesIO(uploaded['NEULAND.xlsx']))
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,0,NaN,740,844.0,NaN
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,0,NaN,844,952.0,740.0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,0,NaN,952,936.0,844.0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,0,NaN,936,1574.0,952.0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,0,NaN,1574,1377.0,936.0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,0,NaN,1377,1180.0,1574.0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,0,NaN,1180,852.0,1377.0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,0,NaN,852,718.0,1180.0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,0,NaN,718,566.0,852.0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,0,NaN,566,581.0,718.0


In [ ]:
#drive.mount('/content/drive', force_remount=True)

In [ ]:
#from tabula import read_pdf
#from tabula import wrapper
#df = wrapper.read_pdf('my_pdf')
#df = wrapper.read_pdf('/content/drive/MyDrive/concall/Neuland-Q3-FY21.pdf')
#dfs = tabula.read_pdf('/content/drive/MyDrive/concall/Neuland-Q4-FY21.pdf', pages="all", stream = True)
#dfs[0]
# convert PDF into CSV file
#tabula.convert_into('/content/drive/MyDrive/concall/Neuland-Q4-FY21.pdf', "/content/drive/MyDrive/concall/output.csv", output_format="csv", pages='all')

In [ ]:
#Use if not importing data frame excel
# assign data of lists.  
#data = {'PDF': ['1 Neuland-Q3-FY21.pdf', '2 Neuland-Q4-FY21.pdf','3 Neuland-Q1-FY22.pdf','4 Neuland-Q2-FY22.pdf','5 Neuland-Q3-FY22.pdf'],'Company': ['Neuland', 'Neuland','Neuland', 'Neuland','Neuland'],'Sector': ['Pharma', 'Pharma','Pharma', 'Pharma','Pharma'], 'Year': ['2021', '2021','2022','2022','2022'],'Quarter': ['Q3', 'Q4','Q1','Q2','Q3'],'Page_Count': [0,0,0,0,0],'Content': ['','','','',''],'Stock_price': [1221,2425,2200,1823,1690]}  
# Create DataFrame  
#concall = pd.DataFrame(data)    

In [5]:
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,0,NaN,740,844.0,NaN
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,0,NaN,844,952.0,740.0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,0,NaN,952,936.0,844.0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,0,NaN,936,1574.0,952.0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,0,NaN,1574,1377.0,936.0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,0,NaN,1377,1180.0,1574.0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,0,NaN,1180,852.0,1377.0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,0,NaN,852,718.0,1180.0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,0,NaN,718,566.0,852.0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,0,NaN,566,581.0,718.0


In [ ]:
# Sample code already present in for loop
#pdfFileObj = open('/content/drive/MyDrive/concall/Neuland-Q3-FY21.pdf', 'rb')
#pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
# printing number of pages in pdf file
#page_count = pdfReader.numPages
# creating a page object
#pageObj = pdfReader.getPage(2)
# extracting text from page
#df = pageObj.extractText()
#df1 = ""
#for page in range(pdfReader.numPages):
#        pageObj = pdfReader.getPage(page)
#        df = pageObj.extractText()
#        df1 = df1+df
# closing the pdf file object
#pdfFileObj.close()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
mypath = '/content/drive/MyDrive/concall/'
from os import listdir
from os.path import isfile, join
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
length = len(files)

In [15]:
pdfFileObj = open(mypath+files[0], 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
page_count = len(pdfReader.pages)
page_count
pageObj = pdfReader.pages[10]
pageObj

{'/Annots': [IndirectObject(341, 0, 139658961557968)],
 '/Type': '/Page',
 '/Tabs': '/S',
 '/MediaBox': [0, 0, 595.44, 841.68],
 '/Parent': {'/Type': '/Pages',
  '/Count': 19,
  '/Kids': [IndirectObject(3, 0, 139658961557968),
   IndirectObject(24, 0, 139658961557968),
   IndirectObject(30, 0, 139658961557968),
   IndirectObject(32, 0, 139658961557968),
   IndirectObject(34, 0, 139658961557968),
   IndirectObject(36, 0, 139658961557968),
   IndirectObject(38, 0, 139658961557968),
   IndirectObject(40, 0, 139658961557968),
   IndirectObject(42, 0, 139658961557968),
   IndirectObject(44, 0, 139658961557968),
   IndirectObject(46, 0, 139658961557968),
   IndirectObject(48, 0, 139658961557968),
   IndirectObject(50, 0, 139658961557968),
   IndirectObject(52, 0, 139658961557968),
   IndirectObject(54, 0, 139658961557968),
   IndirectObject(56, 0, 139658961557968),
   IndirectObject(58, 0, 139658961557968),
   IndirectObject(60, 0, 139658961557968),
   IndirectObject(62, 0, 139658961557968)]

In [16]:
pdfFileObj = open(mypath+files[0], 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
# printing number of pages in pdf file
page_count = len(pdfReader.pages)
# creating a page object
pageObj = pdfReader.pages[0]
pageObj

{'/Type': '/Page',
 '/Tabs': '/S',
 '/MediaBox': [0, 0, 595.44, 841.68],
 '/Parent': {'/Type': '/Pages',
  '/Count': 19,
  '/Kids': [IndirectObject(3, 0, 139658959864352),
   IndirectObject(24, 0, 139658959864352),
   IndirectObject(30, 0, 139658959864352),
   IndirectObject(32, 0, 139658959864352),
   IndirectObject(34, 0, 139658959864352),
   IndirectObject(36, 0, 139658959864352),
   IndirectObject(38, 0, 139658959864352),
   IndirectObject(40, 0, 139658959864352),
   IndirectObject(42, 0, 139658959864352),
   IndirectObject(44, 0, 139658959864352),
   IndirectObject(46, 0, 139658959864352),
   IndirectObject(48, 0, 139658959864352),
   IndirectObject(50, 0, 139658959864352),
   IndirectObject(52, 0, 139658959864352),
   IndirectObject(54, 0, 139658959864352),
   IndirectObject(56, 0, 139658959864352),
   IndirectObject(58, 0, 139658959864352),
   IndirectObject(60, 0, 139658959864352),
   IndirectObject(62, 0, 139658959864352)]},
 '/Resources': {'/Font': {'/F1': {'/Type': '/Font',


In [17]:
for i in range(length):
  pdfFileObj = open(mypath+files[i], 'rb')
  pdfReader = PyPDF2.PdfReader(pdfFileObj)
  # printing number of pages in pdf file
  page_count = len(pdfReader.pages)
  # creating a page object
  pageObj = pdfReader.pages[0]
  # extracting text from page
  df = pageObj.extract_text()
  df1 = ""
  for page in range(page_count):
        pageObj = pdfReader.pages[page]
        df = pageObj.extract_text()
        df1 = df1+df
        concall['PAGE_COUNT'][i] = page_count
        concall['CONTENT'][i] = df1
# closing the pdf file object
pdfFileObj.close()

<ipython-input-17-bc76dcfcdfa3>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concall['PAGE_COUNT'][i] = page_count
<ipython-input-17-bc76dcfcdfa3>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concall['CONTENT'][i] = df1


In [18]:
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,19,\n Page 1 of 19 \n \n \n“Neuland Laboratorie...,740,844.0,NaN
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,20,\n Page 1 of 20 \n \n \n“Neuland Laboratorie...,844,952.0,740.0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,24,\n Page 1 of 24 \n \n \n“Neuland Laboratorie...,952,936.0,844.0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,\n Page 1 of 15 \n \n \n“Neuland Laboratorie...,936,1574.0,952.0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,16,\n Page 1 of 16 \n \n \n“Neuland Laboratorie...,1574,1377.0,936.0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,16,\n Page 1 of 16 \n \n \n“Neuland Laboratorie...,1377,1180.0,1574.0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,18,\n Page 1 of 18 \n \n \n“Neuland Laboratorie...,1180,852.0,1377.0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,19,\n Page 1 of 19 \n \n \n“Neuland Laboratorie...,852,718.0,1180.0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,10,\n Page 1 of 10 \n \n“Neuland Laboratories L...,718,566.0,852.0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,15,\n Page 1 of 15 \n \n \n“Neuland Laboratorie...,566,581.0,718.0


In [19]:
def word_count(text_string):
  return len(text_string.split())

In [20]:
concall['WORD_COUNT'] = concall['CONTENT'].apply(word_count)
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,19,\n Page 1 of 19 \n \n \n“Neuland Laboratorie...,740,844.0,NaN,8487
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,20,\n Page 1 of 20 \n \n \n“Neuland Laboratorie...,844,952.0,740.0,9300
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,24,\n Page 1 of 24 \n \n \n“Neuland Laboratorie...,952,936.0,844.0,11665
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,\n Page 1 of 15 \n \n \n“Neuland Laboratorie...,936,1574.0,952.0,7912
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,16,\n Page 1 of 16 \n \n \n“Neuland Laboratorie...,1574,1377.0,936.0,8613
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,16,\n Page 1 of 16 \n \n \n“Neuland Laboratorie...,1377,1180.0,1574.0,8340
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,18,\n Page 1 of 18 \n \n \n“Neuland Laboratorie...,1180,852.0,1377.0,8897
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,19,\n Page 1 of 19 \n \n \n“Neuland Laboratorie...,852,718.0,1180.0,10236
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,10,\n Page 1 of 10 \n \n“Neuland Laboratories L...,718,566.0,852.0,4681
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,15,\n Page 1 of 15 \n \n \n“Neuland Laboratorie...,566,581.0,718.0,7527


In [21]:
#Identify and remove common words
freq = pd.Series(' '.join(concall['CONTENT']).split()).value_counts()[:20]
freq
freq = list(freq.index)
concall['CONTENT'] = concall['CONTENT'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
concall['CONTENT'].head()

0    Page 1 19 “Neuland Laboratories Limited's Q3 F...
1    Page 1 20 “Neuland Laboratories Limited's Q2 F...
2    Page 1 24 “Neuland Laboratories Limited's Q1 F...
3    Page 1 15 “Neuland Laboratories Limited Q3 FY2...
4    Page 1 16 “Neuland Laboratories Limited's Q4 F...
Name: CONTENT, dtype: object

In [22]:
#Remove rare words
freq = pd.Series(' '.join(concall['CONTENT']).split()).value_counts()[-10:]
freq
freq = list(freq.index)
concall['CONTENT'] = concall['CONTENT'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
concall['CONTENT'].head()

0    Page 1 19 “Neuland Laboratories Limited's Q3 F...
1    Page 1 20 “Neuland Laboratories Limited's Q2 F...
2    Page 1 24 “Neuland Laboratories Limited's Q1 F...
3    Page 1 15 “Neuland Laboratories Limited Q3 FY2...
4    Page 1 16 “Neuland Laboratories Limited's Q4 F...
Name: CONTENT, dtype: object

In [23]:
#Spelling correction - will take longer time to run. Avoid
#from textblob import TextBlob
#concall['Content'].apply(lambda x: str(TextBlob(x).correct()))

In [24]:
#Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
concall['CONTENT'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0     page 1 19 “neuland laboratori limited' q3 fy22...
1     page 1 20 “neuland laboratori limited' q2 fy22...
2     page 1 24 “neuland laboratori limited' q1 fy 2...
3     page 1 15 “neuland laboratori limit q3 fy2 1 e...
4     page 1 16 “neuland laboratori limited' q4 fy'2...
5     page 1 16 “neuland laboratori limit q2 fy21 ea...
6     page 1 18 “neuland laboratori limit q1 fy-21 e...
7     page 1 19 “neuland laboratori limit q 4 fy20 e...
8     page 1 10 “neuland laboratori limit q3 fy20 ea...
9     page 1 15 “neuland laboratori limit q2 fy20 ea...
10    page 1 14 “neuland laboratori q1 fy20 earn con...
11    page 1 14 “neuland laboratori limit q4 fy 2019...
12    page 1 13 “neuland laboratori limit q3 fy19 ea...
13    page 1 12 “neuland laboratori limit q2 fy19 ea...
14    page 1 17 “neuland laboratori limit q1 fy19 ea...
15    page 1 24 “neuland laboratori limit q4 fy18 ea...
16    page 1 16 “neuland laboratori q2 fy18 confer c...
17    page 1 13 “neuland laboratori limit q1 fy1

In [25]:
concall['WORD_COUNT'] = concall['CONTENT'].apply(word_count)
concall


,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,19,Page 1 19 “Neuland Laboratories Limited's Q3 F...,740,844.0,NaN,5932
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,20,Page 1 20 “Neuland Laboratories Limited's Q2 F...,844,952.0,740.0,6565
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,24,Page 1 24 “Neuland Laboratories Limited's Q1 F...,952,936.0,844.0,8057
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q3 FY2...,936,1574.0,952.0,5415
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,16,Page 1 16 “Neuland Laboratories Limited's Q4 F...,1574,1377.0,936.0,6000
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,1377,1180.0,1574.0,5599
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,18,Page 1 18 “Neuland Laboratories Limited Q1 FY-...,1180,852.0,1377.0,6195
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,19,Page 1 19 “Neuland Laboratories Limited Q 4 FY...,852,718.0,1180.0,6960
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,718,566.0,852.0,3233
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,566,581.0,718.0,5049


In [26]:
#count stopwords
import nltk
nltk.download('all')
from nltk.corpus import stopwords
print(stopwords.words('english'))


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]    |   Unzipping corpora/ycoe.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection all


In [27]:
stop_words = set(stopwords.words('english'))
concall['STOPWORDS_COUNT'] = concall['CONTENT'].apply(lambda x: len([x for x in x.split() if x in stop_words]))
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT,STOPWORDS_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,19,Page 1 19 “Neuland Laboratories Limited's Q3 F...,740,844.0,NaN,5932,1164
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,20,Page 1 20 “Neuland Laboratories Limited's Q2 F...,844,952.0,740.0,6565,1330
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,24,Page 1 24 “Neuland Laboratories Limited's Q1 F...,952,936.0,844.0,8057,1666
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q3 FY2...,936,1574.0,952.0,5415,1024
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,16,Page 1 16 “Neuland Laboratories Limited's Q4 F...,1574,1377.0,936.0,6000,1171
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,1377,1180.0,1574.0,5599,1243
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,18,Page 1 18 “Neuland Laboratories Limited Q1 FY-...,1180,852.0,1377.0,6195,1250
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,19,Page 1 19 “Neuland Laboratories Limited Q 4 FY...,852,718.0,1180.0,6960,1395
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,718,566.0,852.0,3233,629
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,566,581.0,718.0,5049,924


In [28]:
#Remove stopwords
concall['CONTENT'] = concall['CONTENT'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
concall['WORD_COUNT'] = concall['CONTENT'].apply(word_count)
concall['STOPWORDS_COUNT'] = concall['CONTENT'].apply(lambda x: len([x for x in x.split() if x in stop_words]))
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT,STOPWORDS_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,19,Page 1 19 “Neuland Laboratories Limited's Q3 F...,740,844.0,NaN,4768,0
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,20,Page 1 20 “Neuland Laboratories Limited's Q2 F...,844,952.0,740.0,5235,0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,24,Page 1 24 “Neuland Laboratories Limited's Q1 F...,952,936.0,844.0,6391,0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q3 FY2...,936,1574.0,952.0,4391,0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,16,Page 1 16 “Neuland Laboratories Limited's Q4 F...,1574,1377.0,936.0,4829,0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,1377,1180.0,1574.0,4356,0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,18,Page 1 18 “Neuland Laboratories Limited Q1 FY-...,1180,852.0,1377.0,4945,0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,19,Page 1 19 “Neuland Laboratories Limited Q 4 FY...,852,718.0,1180.0,5565,0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,718,566.0,852.0,2604,0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,566,581.0,718.0,4125,0


In [29]:
concall['CONTENT'][0]

"Page 1 19 “Neuland Laboratories Limited's Q3 FY22 Earnings Conference Call” February 1, 2022 MANAGEMENT : MR. SUCHETH DAVULURI - VICE CHAIRMAN & CEO , NEULAND LABORATORIES LIMITED MR. SAHARSH DAVULURI – VICE CHAIRMAN & MANAGING DIRECTOR , NEULAND LABORATORIES LIMITED MR. DEEPAK GUPTA – CFO, NEULAND LABORATORIES LIMITED MR. SAJEEV EMMANUEL MEDIKONDA – HEAD, CORPORATE PLANNING & STRATEGY , NEULAND LABORATORIES LIMITED Neuland Laboratories Limited February 1, 2022 Page 2 19 Ravi Udeshi : Good evening friends. We welcome Q3 FY22 earnings call Neuland Laboratories Limited. To take us results answer questions, us top management Neuland, represented Mr. Sucheth Davuluri – Vice Chairman CEO ; Mr. Saharsh Davuluri – Vice Chairman Managing Director; Mr. Deepak Gupta – CFO Mr. Sajeev Emmanuel Medikonda - Head, Corporate Planning Strategy. We start call brief overview financials Deepak, Saharsh give br oad highlight business trends observing market open floor Q&A . With standard safe harbor claus

In [30]:
analyzer = SentimentIntensityAnalyzer()
def sentiment_analyzser_scores(text):
  score = analyzer.polarity_scores(text)
  print(text)
  print(score)

In [31]:
text_pos = 'it could have been better'
sentiment_analyzser_scores(text_pos)

it could have been better
{'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compound': 0.4404}


In [32]:
text_pos = concall['CONTENT'][1]
sentiment_analyzser_scores(text_pos)

Page 1 20 “Neuland Laboratories Limited's Q2 FY22 Earnings Conference Call” October 29, 2021 MANAGEMENT : MR. SAHARSH DAVULURI – VICE CHAIRMAN & MANAGING DIRECTOR , NEULAND LABORATORIES LIMITED MR. DEEPAK GUPTA – CFO, NEULAND LABORATORIES LIMITED MR SUNDAR NARSHIMAN – SENIOR VICE PRESIDENT SUPPLY CHAIN , NEULAND LABORATORIES LIMITED MR. SAJEEV EMMANUEL MEDIKONDA – HEAD, CORPORATE PLANNING & STRATEGY , NEULAND LABORATORIES LIMITED Neuland Laboratories Limited October 29 , 2021 Page 2 20 Diwakar Pingle : Good evening friends. We welcome Q2 FY22 earnings call Neuland Laboratories Limited. To take us results answer questions, top management Neuland, represented Mr. Saharsh Davuluri – Vice Chairman Managing Director; Mr. Deepak Gupta – CFO ; Mr Sundar Narshiman – Senior Vice President Supply Chain Mr. Sajeev Emmanuel Medikonda - Head, Corporate Planning Strategy. We start call brief overview financials Deepak, Saharsh give broad highlight general business trends observing market open floor 

In [33]:
def get_pos_word(x):
  text = x['CONTENT']
  tokenized_text = nltk.word_tokenize(text)
  pos_word_list =[]
  neu_word_list=[]
  neg_word_list=[]
  for word in tokenized_text:
    if(analyzer.polarity_scores(word)['compound'])>=0.5:
      pos_word_list.append(word)
    elif (analyzer.polarity_scores(word)['compound']) <=-0.5:
      neg_word_list.append(word)
    else:
      neu_word_list.append(word)
  return set(pos_word_list)

In [34]:
def get_neg_word(x):
  text = x['CONTENT']
  tokenized_text = nltk.word_tokenize(text)
  pos_word_list =[]
  neu_word_list=[]
  neg_word_list=[]
  for word in tokenized_text:
    if(analyzer.polarity_scores(word)['compound'])>=0.5:
      pos_word_list.append(word)
    elif (analyzer.polarity_scores(word)['compound']) <=-0.5:
      neg_word_list.append(word)
    else:
      neu_word_list.append(word)
  return set(neg_word_list)

In [35]:
from afinn import Afinn
afinn = Afinn(language = 'en')

In [36]:
concall['AFINN_SCORE'] = concall['CONTENT'].apply(afinn.score)

In [37]:
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT,STOPWORDS_COUNT,AFINN_SCORE
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,19,Page 1 19 “Neuland Laboratories Limited's Q3 F...,740,844.0,NaN,4768,0,481.0
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,20,Page 1 20 “Neuland Laboratories Limited's Q2 F...,844,952.0,740.0,5235,0,555.0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,24,Page 1 24 “Neuland Laboratories Limited's Q1 F...,952,936.0,844.0,6391,0,693.0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q3 FY2...,936,1574.0,952.0,4391,0,460.0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,16,Page 1 16 “Neuland Laboratories Limited's Q4 F...,1574,1377.0,936.0,4829,0,359.0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,1377,1180.0,1574.0,4356,0,502.0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,18,Page 1 18 “Neuland Laboratories Limited Q1 FY-...,1180,852.0,1377.0,4945,0,504.0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,19,Page 1 19 “Neuland Laboratories Limited Q 4 FY...,852,718.0,1180.0,5565,0,504.0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,718,566.0,852.0,2604,0,226.0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,566,581.0,718.0,4125,0,464.0


In [38]:
concall['AFINN_ADJUSTED'] = concall['AFINN_SCORE']/concall['WORD_COUNT'] * 100

In [39]:
sentiment = concall['CONTENT'].apply(analyzer.polarity_scores)

In [40]:
sentiment_df = pd.DataFrame(sentiment.tolist())
sentiment_df

,neg,neu,pos,compound
0,0.029,0.765,0.207,1.0000
1,0.029,0.765,0.206,1.0000
2,0.032,0.760,0.208,1.0000
3,0.039,0.757,0.204,1.0000
4,0.032,0.789,0.180,1.0000
5,0.034,0.761,0.205,1.0000
6,0.040,0.736,0.224,1.0000
7,0.036,0.755,0.209,1.0000
8,0.038,0.773,0.190,0.9999
9,0.032,0.773,0.196,1.0000


In [41]:
df_sentiment = pd.concat([concall,sentiment_df], axis = 1)

In [42]:
df_sentiment

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT,STOPWORDS_COUNT,AFINN_SCORE,AFINN_ADJUSTED,neg,neu,pos,compound
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,19,Page 1 19 “Neuland Laboratories Limited's Q3 F...,740,844.0,NaN,4768,0,481.0,10.088087,0.029,0.765,0.207,1.0000
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,20,Page 1 20 “Neuland Laboratories Limited's Q2 F...,844,952.0,740.0,5235,0,555.0,10.601719,0.029,0.765,0.206,1.0000
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,24,Page 1 24 “Neuland Laboratories Limited's Q1 F...,952,936.0,844.0,6391,0,693.0,10.843373,0.032,0.760,0.208,1.0000
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q3 FY2...,936,1574.0,952.0,4391,0,460.0,10.475974,0.039,0.757,0.204,1.0000
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,16,Page 1 16 “Neuland Laboratories Limited's Q4 F...,1574,1377.0,936.0,4829,0,359.0,7.434251,0.032,0.789,0.180,1.0000
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,1377,1180.0,1574.0,4356,0,502.0,11.524334,0.034,0.761,0.205,1.0000
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,18,Page 1 18 “Neuland Laboratories Limited Q1 FY-...,1180,852.0,1377.0,4945,0,504.0,10.192113,0.040,0.736,0.224,1.0000
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,19,Page 1 19 “Neuland Laboratories Limited Q 4 FY...,852,718.0,1180.0,5565,0,504.0,9.056604,0.036,0.755,0.209,1.0000
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,718,566.0,852.0,2604,0,226.0,8.678955,0.038,0.773,0.190,0.9999
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,566,581.0,718.0,4125,0,464.0,11.248485,0.032,0.773,0.196,1.0000


In [43]:
df_sentiment['POS_WORD'] = concall.apply(lambda x:get_pos_word(x),axis=1)

In [44]:
df_sentiment['NEG_WORD'] = concall.apply(lambda x:get_neg_word(x),axis=1)

In [45]:
df_sentiment['POS_COUNT'] = df_sentiment['POS_WORD'].apply(lambda x: len(x))

In [46]:
df_sentiment['NEG_COUNT'] = df_sentiment['NEG_WORD'].apply(lambda x: len(x))

In [47]:
df_sentiment

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,...,AFINN_SCORE,AFINN_ADJUSTED,neg,neu,pos,compound,POS_WORD,NEG_WORD,POS_COUNT,NEG_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,19,Page 1 19 “Neuland Laboratories Limited's Q3 F...,740,844.0,...,481.0,10.088087,0.029,0.765,0.207,1.0000,"{great, Great, best, Awesome, comfortable, kin...","{po, failed}",16,2
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,20,Page 1 20 “Neuland Laboratories Limited's Q2 F...,844,952.0,...,555.0,10.601719,0.029,0.765,0.206,1.0000,"{great, best, congratulation, kind, alol, succ...","{disappointment, bad, fu, stressful}",13,4
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,24,Page 1 24 “Neuland Laboratories Limited's Q1 F...,952,936.0,...,693.0,10.843373,0.032,0.760,0.208,1.0000,"{great, best, won, kind, optimism, free, ly, p...","{rage, stab}",17,2
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q3 FY2...,936,1574.0,...,460.0,10.475974,0.039,0.757,0.204,1.0000,"{great, comfortable, kind, wonderful, positive...",{failed},14,1
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,16,Page 1 16 “Neuland Laboratories Limited's Q4 F...,1574,1377.0,...,359.0,7.434251,0.032,0.789,0.180,1.0000,"{best, kind, free, gain, trust, strong, succes...","{Distress, Depression}",9,2
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,1377,1180.0,...,502.0,11.524334,0.034,0.761,0.205,1.0000,"{great, best, kind, optimism, free, positive, ...",{worst},13,1
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,18,Page 1 18 “Neuland Laboratories Limited Q1 FY-...,1180,852.0,...,504.0,10.192113,0.040,0.736,0.224,1.0000,"{great, amazing, kind, successful, trust, bloc...","{degradation, threat, spite}",10,3
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,19,Page 1 19 “Neuland Laboratories Limited Q 4 FY...,852,718.0,...,504.0,9.056604,0.036,0.755,0.209,1.0000,"{comfortable, best, won, kind, successful, tru...","{threat, PO, bad, devil, disaster}",14,5
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,718,566.0,...,226.0,8.678955,0.038,0.773,0.190,0.9999,"{comfortable, kind, positive, encouraging, str...","{failure, bad}",7,2
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,566,581.0,...,464.0,11.248485,0.032,0.773,0.196,1.0000,"{great, comfortable, best, kind, optimism, pos...","{worst, negative}",11,2


In [50]:
df_sentiment.drop(['CONTENT'], axis =1, inplace = True)

In [51]:
from google.colab import files
df_sentiment.to_csv("sentiment_analysis.csv",encoding="utf_8")
files.download('sentiment_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>